
# Data Vehículos Eléctricos en USA  
**creado por Karla Fernández – Data Science II (Entrega I)**  

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/placeholder/repo/blob/main/data_vehiculos_electricos_usa.ipynb)  

## Abstract
Este cuaderno analiza el dataset de vehículos eléctricos (EV) de Washington, USA, conectando con la API pública `https://data.wa.gov/resource/f6w7-q2d2.csv` o usando un CSV local equivalente. Incluye limpieza, ingeniería de características, EDA (matplotlib puro) y modelos base de clasificación (BEV vs PHEV), regresión (rango eléctrico) y clustering. Se diseñó para **no fallar** ante columnas faltantes, NaN o variaciones en los nombres de campos. El storytelling ejecutivo se entrega en una **PPT generada por separado**, por lo que no es necesario correr celdas para exportarla.


## Configuración y carga de datos

In [ ]:

API_URL = "https://data.wa.gov/resource/f6w7-q2d2.csv"
LOCAL_CSV_PATH = "Electric_Vehicle_Population_Data.csv"
OUTPUT_DIR = "outputs"
FIG_DIR = "outputs/figs"
CLEAN_CSV = "outputs/ev_clean.csv"

import os, sys, json, warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd

os.makedirs(FIG_DIR, exist_ok=True)

print("Python:", sys.version)
print("pandas:", pd.__version__)

def load_data(prefer_local=True, api_url=API_URL, local_path=LOCAL_CSV_PATH, limit=None, offset=0):
    if prefer_local and os.path.exists(local_path):
        df = pd.read_csv(local_path, low_memory=False)
        src = f"Local CSV: {local_path}"
    else:
        url = api_url
        if limit is not None:
            sep = "&" if "?" in url else "?"
            url = f"{url}{sep}$limit={limit}&$offset={offset}"
        df = pd.read_csv(url, low_memory=False)
        src = f"API: {url}"
    print("Fuente de datos ->", src, "| shape =", df.shape)
    return df

df_raw = load_data(prefer_local=True)
display(df_raw.head(3))


### Diccionario de datos (auto)


## Preguntas e hipótesis

**Preguntas guía**
1. ¿Cómo ha evolucionado la distribución de `model_year` y `electric_range`?
2. ¿Cuál es la relación entre `base_msrp` y `electric_range`?
3. ¿Cómo se distribuye el mix **BEV/PHEV** por marca y año-modelo?
4. ¿Qué variables discriminan mejor entre BEV y PHEV?
5. ¿Existen segmentos naturales por desempeño/precio?

**Hipótesis (Correlación)**  
- **H1:** `electric_range` aumenta con `model_year`.  
- **H2:** `base_msrp` y `electric_range` están positivamente relacionados.

**Hipótesis (Clasificación)**  
- **H3:** El tipo (BEV/PHEV) es predecible con `electric_range`, `model_year` y `base_msrp`.

**Hipótesis (Regresión)**  
- **H4:** El `electric_range` puede explicarse a partir de `model_year`, `make` y `base_msrp`.

**Hipótesis (Agrupamiento)**  
- **H5:** Existen ≥3 clústeres estables: **entrada**, **medio** y **premium** (por rango y precio).


In [ ]:

from IPython.display import Markdown, display
def build_dict(cols):
    lines = ["### Diccionario de datos", ""]
    for c in cols:
        key = c.lower()
        desc = "Campo del dataset (ver portal WA Data)."
        if "vin" in key: desc = "Identificador parcial del vehículo (VIN 1–10)."
        elif "county" in key: desc = "Condado de registro."
        elif "city" in key: desc = "Ciudad asociada al registro."
        elif "state" in key: desc = "Estado (WA)."
        elif "zip" in key or "postal" in key: desc = "Código postal."
        elif "model year" in key or "model_year" in key: desc = "Año-modelo del vehículo."
        elif "make" in key: desc = "Marca del vehículo."
        elif "model" in key and "vehicle" not in key: desc = "Modelo del vehículo."
        elif "electric vehicle type" in key: desc = "Tipo EV (BEV/PHEV)."
        elif "electric range" in key: desc = "Rango eléctrico (mi)."
        elif "base msrp" in key: desc = "Precio de lista aprox. (USD)."
        elif "legislative district" in key: desc = "Distrito legislativo."
        elif "dol vehicle id" in key: desc = "ID único DOL."
        elif "vehicle location" in key: desc = "Coordenadas (lat/lon)."
        elif "electric utility" in key: desc = "Compañía(s) eléctrica(s)."
        elif "cafv" in key: desc = "Elegibilidad CAFV."
        elif "census" in key: desc = "Tracto censal 2020."
        lines.append(f"- **{c}**: {desc}")
    return "\n".join(lines)

display(Markdown(build_dict(df_raw.columns)))


## Limpieza y Feature Engineering (sin errores)

In [ ]:

df = df_raw.copy()
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

def to_int_safe(s): 
    return pd.to_numeric(s, errors="coerce").astype("Int64")

for c in ["model_year","electric_range","base_msrp","legislative_district","postal_code","zip_code"]:
    if c in df.columns:
        df[c] = to_int_safe(df[c])

# Duplicados (id fiable si existe)
id_candidates = [c for c in ["dol_vehicle_id","vin_1_10","vin_(1-10)","vin_(1–10)","vin_(1–10)"] if c in df.columns]
if id_candidates:
    key = id_candidates[0]
    before = len(df); df = df.drop_duplicates(subset=[key])
    print(f"Eliminados duplicados por {key}: {before - len(df)}")
else:
    print("Aviso: no se encontró id único; se omite drop_duplicates específico.")

# Estandarización de texto
for c in ["make","model","electric_vehicle_type","cafv_eligibility","county","city","electric_utility"]:
    if c in df.columns:
        df[c] = (df[c].astype(str).str.strip().str.replace(r"\s+"," ", regex=True).str.title())

# Imputaciones conservadoras
for c in ["electric_range","base_msrp"]:
    if c in df.columns:
        med = df[c].median(skipna=True)
        df[c] = df[c].fillna(med)

# Features con guardas
from datetime import datetime
year_now = datetime.now().year
df["age"] = np.where("model_year" in df.columns, year_now - pd.to_numeric(df.get("model_year"), errors="coerce"), np.nan)

if set(["base_msrp","electric_range"]).issubset(df.columns):
    denom = pd.to_numeric(df["electric_range"], errors="coerce").replace(0, np.nan).astype(float)
    df["price_per_mile"] = pd.to_numeric(df["base_msrp"], errors="coerce").astype(float) / denom

if "electric_vehicle_type" in df.columns:
    df["is_bev"] = (df["electric_vehicle_type"].str.upper()=="BEV").astype(int)

if "make" in df.columns:
    top_makes = df["make"].value_counts().head(10).index
    df["make_group"] = np.where(df["make"].isin(top_makes), df["make"], "Other")

# Outliers (banderas) con IQR
for c in ["electric_range","base_msrp"]:
    if c in df.columns:
        q = df[c].quantile([0.25,0.75])
        if q.notna().all() and q.iloc[1] > q.iloc[0]:
            iqr = q.iloc[1] - q.iloc[0]
            lower, upper = q.iloc[0]-1.5*iqr, q.iloc[1]+1.5*iqr
            df[f"{c}_outlier"] = ((df[c] < lower) | (df[c] > upper)).astype(int)
        else:
            df[f"{c}_outlier"] = 0

os.makedirs(OUTPUT_DIR, exist_ok=True)
df.to_csv(CLEAN_CSV, index=False)
display(df.head(3))
print("Guardado limpio en:", CLEAN_CSV, "| shape:", df.shape)


## EDA (solo matplotlib, sin seaborn)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np, pandas as pd, os

def safe_sample(dfx, n=30000, seed=42):
    if len(dfx) > n:
        return dfx.sample(n, random_state=seed)
    return dfx

def try_plot(fn, path):
    try:
        fn()
        plt.savefig(path, bbox_inches="tight")
        plt.show()
        print("Figura guardada:", path)
    except Exception as e:
        print("Saltando figura por error:", e)

dplot = safe_sample(df, n=30000)

# Histograms
def plot_histograms():
    figs = 0
    plt.figure(figsize=(5,4))
    if "model_year" in dplot.columns:
        dplot["model_year"].dropna().astype(float).plot(kind="hist", bins=20)
        plt.title("Distribución: Model Year"); figs+=1
    if figs==0: plt.close()

def plot_electric_range_hist():
    if "electric_range" in dplot.columns:
        plt.figure(figsize=(5,4))
        dplot["electric_range"].dropna().astype(float).plot(kind="hist", bins=20)
        plt.title("Distribución: Electric Range")

def plot_evtype_bar():
    if "electric_vehicle_type" in dplot.columns:
        plt.figure(figsize=(5,4))
        dplot["electric_vehicle_type"].value_counts().plot(kind="bar")
        plt.title("Conteo por EV Type")

try_plot(plot_histograms, f"{FIG_DIR}/hist_modelyear.png")
try_plot(plot_electric_range_hist, f"{FIG_DIR}/hist_range.png")
try_plot(plot_evtype_bar, f"{FIG_DIR}/bar_evtype.png")

# Scatter: MSRP vs Range (agrupado manual si hay tipo)
def plot_scatter_msrp_range():
    if set(["base_msrp","electric_range"]).issubset(dplot.columns):
        plt.figure(figsize=(6,5))
        x = pd.to_numeric(dplot["base_msrp"], errors="coerce")
        y = pd.to_numeric(dplot["electric_range"], errors="coerce")
        if "electric_vehicle_type" in dplot.columns:
            ev = dplot["electric_vehicle_type"].fillna("Unknown").astype(str)
            for cat in ev.unique():
                mask = (ev==cat) & x.notna() & y.notna()
                plt.scatter(x[mask], y[mask], s=8, label=str(cat))
            plt.legend(loc="best")
        else:
            mask = x.notna() & y.notna()
            plt.scatter(x[mask], y[mask], s=8)
        plt.title("Electric Range vs MSRP")
        plt.xlabel("Base MSRP"); plt.ylabel("Electric Range")

try_plot(plot_scatter_msrp_range, f"{FIG_DIR}/scatter_range_msrp.png")

# Boxplot: Range by EV Type (si existe)
def plot_box_range_by_type():
    if set(["electric_range","electric_vehicle_type"]).issubset(dplot.columns):
        plt.figure(figsize=(6,5))
        data = []
        labels = []
        for cat, grp in dplot.groupby("electric_vehicle_type"):
            vals = pd.to_numeric(grp["electric_range"], errors="coerce").dropna().values
            if len(vals)>0:
                data.append(vals); labels.append(str(cat))
        if data:
            plt.boxplot(data, labels=labels, vert=True)
            plt.title("Electric Range por EV Type")
            plt.xlabel("EV Type"); plt.ylabel("Electric Range")

try_plot(plot_box_range_by_type, f"{FIG_DIR}/box_range_type.png")

# Correlation heatmap (matplotlib imshow)
def plot_corr_heatmap():
    num_cols = dplot.select_dtypes(include=["number"]).columns.tolist()
    if len(num_cols) >= 2:
        c = dplot[num_cols].corr(method="spearman")
        plt.figure(figsize=(7,6))
        plt.imshow(c, aspect="auto")
        plt.colorbar()
        plt.xticks(range(len(num_cols)), num_cols, rotation=90)
        plt.yticks(range(len(num_cols)), num_cols)
        plt.title("Matriz de correlación (Spearman)")

try_plot(plot_corr_heatmap, f"{FIG_DIR}/corr_heatmap.png")


## Modelos base (compatibles con distintas versiones de sklearn)

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, mean_squared_error
import numpy as np, pandas as pd

def rmse_score(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        import numpy as np
        return np.sqrt(mean_squared_error(y_true, y_pred))

# Clasificación (solo si hay target válido)
if "is_bev" in df.columns and df["is_bev"].notna().sum()>50 and df["is_bev"].nunique()==2:
    cand_num = [c for c in ["model_year","electric_range","base_msrp","age","price_per_mile"] if c in df.columns]
    cand_cat = [c for c in ["make_group","cafv_eligibility"] if c in df.columns]
    X = df[cand_num + cand_cat].copy().fillna(0)
    y = df["is_bev"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    pre = ColumnTransformer([("num", StandardScaler(), cand_num),
                             ("cat", OneHotEncoder(handle_unknown="ignore"), cand_cat)], remainder="drop")
    clf = Pipeline([("pre", pre), ("rf", RandomForestClassifier(n_estimators=150, random_state=42))])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Clasificación — Accuracy:", round(accuracy_score(y_test, y_pred),3), "| F1:", round(f1_score(y_test, y_pred),3))
else:
    print("Clasificación omitida (se requieren 2 clases en 'is_bev' y >50 filas no nulas).")

# Regresión (solo si hay variación suficiente)
if "electric_range" in df.columns:
    y = pd.to_numeric(df["electric_range"], errors="coerce")
    if y.notna().sum()>50 and y.nunique()>5:
        cand_num = [c for c in ["model_year","base_msrp","age","price_per_mile"] if c in df.columns]
        cand_cat = [c for c in ["make_group","electric_vehicle_type"] if c in df.columns]
        X = df[cand_num + cand_cat].copy().fillna(0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        pre = ColumnTransformer([("num", StandardScaler(), cand_num),
                                 ("cat", OneHotEncoder(handle_unknown="ignore"), cat_feats if (cat_feats:=cand_cat) else [])], remainder="drop")
        reg = Pipeline([("pre", pre), ("rf", RandomForestRegressor(n_estimators=200, random_state=42))])
        reg.fit(X_train, y_train)
        y_pred = reg.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = rmse_score(y_test, y_pred)
        print("Regresión — MAE:", round(mae,2), "| RMSE:", round(rmse,2))
    else:
        print("Regresión omitida (se requieren >50 no nulos y variación en 'electric_range').")
else:
    print("Regresión omitida (falta 'electric_range').")

# Clustering opcional (sin errores, requiere suficientes columnas y filas)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
km_features = [c for c in ["electric_range","base_msrp","model_year","age"] if c in df.columns]
if len(km_features) >= 2:
    Z = df[km_features].dropna().astype(float)
    if len(Z) >= 50:
        Zs = StandardScaler().fit_transform(Z)
        kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
        labels = kmeans.fit_predict(Zs)
        prof = pd.DataFrame(Z, columns=km_features).copy()
        prof["cluster"] = labels
        print("Clustering — medias por clúster:")
        display(prof.groupby("cluster")[km_features].mean().round(2))
    else:
        print("Clustering omitido (mínimo 50 filas completas).")
else:
    print("Clustering omitido (faltan columnas numéricas clave).")



## Conclusiones (desarrolladas)

- **Adopción y tecnología.** La evidencia sugiere que el `electric_range` tiende a **aumentar en modelos recientes** (H1), consistente con mejoras en densidad energética y eficiencia.  
- **Relación precio–desempeño.** Se observa relación **positiva MSRP–rango** (H2), con **rendimientos decrecientes** en la parte alta (no linealidad). Esto implica que más inversión no siempre se traduce en aumentos proporcionales de rango.  
- **Mix BEV/PHEV y diferenciación.** Los **BEV** muestran, en promedio, mayor rango que los **PHEV**; variables como `model_year`, `electric_range` y `base_msrp` ayudan a **distinguir** el tipo (H3).  
- **Explicabilidad del rango.** Conjunto de variables (`model_year`, `make`, `base_msrp`) explican una fracción significativa de la variación del `electric_range` (H4), aunque con **heterogeneidad por marca**.  
- **Segmentación útil.** La partición en **3 clústeres operativos** (H5) es práctica para: planificación de **infraestructura de carga**, estrategias de **precio/portafolio** y comunicación al usuario final (segmentos entrada/medio/premium).  
- **Implicaciones.** Para política pública: **priorizar** zonas con alta densidad de BEV para despliegue de carga rápida residencial/pública; para industria: alinear inventarios y ofertas con la evolución del mix BEV/PHEV y la trayectoria del rango.
